In [79]:
import pandas as pd

In [5]:
trans = pd.read_csv('csv_files/proteomics.csv', index_col=0)
trans

,Difference_In_Median_Brca,P_Value_Brca,Difference_In_Median_Endo,P_Value_Endo,Difference_In_Median,P_Value
A2ML1,1.26120,0.165194,-1.10585,0.037044,NaN,NaN
AADAT,1.33200,0.027477,0.18535,0.690613,NaN,NaN
AAGAB,0.09455,0.909622,-0.24350,0.139632,-0.1390,0.039189
AASDHPPT,-0.51445,0.003620,0.10590,0.428884,0.0628,0.973515
AATF,0.16905,0.042271,0.18170,0.915482,-0.0250,0.568821
...,...,...,...,...,...,...
ZNF654,-0.49290,0.048872,0.11860,0.294513,NaN,NaN
ZNF768,0.35120,0.069655,-0.19580,0.661851,-0.3755,0.030999
ZNHIT3,0.01085,0.990265,-0.13895,0.029161,NaN,NaN
ZNRD1,0.18620,0.423039,-0.30475,0.017492,NaN,NaN


In [19]:
trans.loc[trans.index.str.contains('FKH')]

,Difference_In_Median_Brca,P_Value_Brca,Difference_In_Median_Endo,P_Value_Endo,Difference_In_Median,P_Value


In [8]:
#get the gene of interest
df = trans.loc[trans.index == "PIK3C3"]
df.head()

,Difference_In_Median_Brca,P_Value_Brca,Difference_In_Median_Endo,P_Value_Endo,Difference_In_Median,P_Value
PIK3C3,-0.3914,0.14189,0.1252,0.012287,0.0406,0.875261


In [39]:
brca = trans.drop(columns=['Difference_In_Median_Endo', 'P_Value_Endo','Difference_In_Median','P_Value'])
brca['cancer_type'] = "brca"
brca.columns = ['Difference_In_Median', 'P_Value','cancer_type']
brca.head()

,Difference_In_Median,P_Value,cancer_type
A2ML1,1.26120,0.165194,brca
AADAT,1.33200,0.027477,brca
AAGAB,0.09455,0.909622,brca
AASDHPPT,-0.51445,0.003620,brca
AATF,0.16905,0.042271,brca


In [37]:
endo = trans.drop(columns=['Difference_In_Median_Brca', 'P_Value_Brca','Difference_In_Median','P_Value'])
endo['cancer_type'] = "endo"
endo.columns = ['Difference_In_Median', 'P_Value','cancer_type']
endo.head()

,Difference_In_Median,P_Value,cancer_type
A2ML1,-1.10585,0.037044,endo
AADAT,0.18535,0.690613,endo
AAGAB,-0.24350,0.139632,endo
AASDHPPT,0.10590,0.428884,endo
AATF,0.18170,0.915482,endo


In [34]:
colon = trans.drop(columns=['Difference_In_Median_Brca', 'P_Value_Brca','Difference_In_Median_Endo','P_Value_Endo'])
colon['cancer_type'] = "colon"
colon.head()

,Difference_In_Median,P_Value,cancer_type
A2ML1,NaN,NaN,colon
AADAT,NaN,NaN,colon
AAGAB,-0.1390,0.039189,colon
AASDHPPT,0.0628,0.973515,colon
AATF,-0.0250,0.568821,colon


In [45]:
c_and_e = colon.append(endo)
df = c_and_e.append(brca)
df["comparison"] = df.index
df

,Difference_In_Median,P_Value,cancer_type,comparison
A2ML1,NaN,NaN,colon,A2ML1
AADAT,NaN,NaN,colon,AADAT
AAGAB,-0.13900,0.039189,colon,AAGAB
AASDHPPT,0.06280,0.973515,colon,AASDHPPT
AATF,-0.02500,0.568821,colon,AATF
...,...,...,...,...
ZNF654,-0.49290,0.048872,brca,ZNF654
ZNF768,0.35120,0.069655,brca,ZNF768
ZNHIT3,0.01085,0.990265,brca,ZNHIT3
ZNRD1,0.18620,0.423039,brca,ZNRD1


In [49]:
import pandas as pd
import numpy as np
from bokeh.palettes import RdBu
from bokeh.models import LinearColorMapper, ColumnDataSource, ColorBar
from bokeh.models.ranges import FactorRange
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.io import export_png
from bokeh.io import export_svgs
import math as math


'''
@Param df: Dataframe. Contains column with x-axis categorical variables, y-axis categorical variables,
and columns for circle size and color gradient. 
@Param circle_var. String. Name of column for numeric data to base circle size off of 
@Param color_var. String. Name of column of numeric data to base color gradient off of. Can be the same or different as circle_var
@Param x_axis String. Name of column for x-axis categorical labels
@Param y_axis String. Name of column for y-axis categorical labels
@Param x_axis_lab. String. Default is no label. 
@Param y_axis_lab. String. Default is no label. 

This function creates a bokeh map that is heat map with extra variable of size of the circles. 

'''
def plotCircleHeatMap ( df, circle_var, color_var, x_axis, y_axis,x_axis_lab = "no_label", y_axis_lab = "no_label"):
  


    #added a new column to make the plot size
    df['size'] = np.where(df[circle_var]<0, np.abs(df[circle_var]), df[circle_var])*50
 

    colors = list(reversed(RdBu[9]))
    exp_cmap = LinearColorMapper(palette=colors, low = -1, high = 1)
    p = figure(x_range = FactorRange(), y_range = FactorRange(), plot_width=700, 
               plot_height=450, 
               toolbar_location=None, tools="hover")

    p.scatter(x_axis,y_axis,source=df, fill_alpha=1,  line_width=0, size="size", 
              fill_color={"field":color_var, "transform":exp_cmap})

    p.x_range.factors = sorted(df[x_axis].unique().tolist())
    p.y_range.factors = sorted(df[y_axis].unique().tolist(), reverse = True)
    p.xaxis.major_label_orientation = math.pi/2
    
    if (x_axis_lab != "no_label" ):
        p.xaxis.axis_label = x_axis_lab
    if (x_axis_lab != "no_label" ):   
        p.yaxis.axis_label = y_axis_lab

    bar = ColorBar(color_mapper=exp_cmap, location=(0,0))
    p.add_layout(bar, "right")
    output_notebook()
  
    show(p)

In [50]:
plotCircleHeatMap(df,"P_Value","P_Value","comparison","cancer_type")

Loading BokehJS ...

In [51]:
df.head()

,Difference_In_Median,P_Value,cancer_type,comparison,size
A2ML1,NaN,NaN,colon,A2ML1,NaN
AADAT,NaN,NaN,colon,AADAT,NaN
AAGAB,-0.1390,0.039189,colon,AAGAB,1.959434
AASDHPPT,0.0628,0.973515,colon,AASDHPPT,48.675735
AATF,-0.0250,0.568821,colon,AATF,28.441045


In [62]:
selected = df.loc[(df.index=="PIK3C3")
                 |(df.index=="BAD")
                 |(df.index=="AKT2")
                 |(df.index=="BECN1")]
selected

,Difference_In_Median,P_Value,cancer_type,comparison,size
AKT2,0.080600,0.942103,colon,AKT2,47.105137
BAD,-0.079400,0.205739,colon,BAD,10.286963
BECN1,-0.123600,0.294600,colon,BECN1,14.729995
PIK3C3,0.040600,0.875261,colon,PIK3C3,43.763072
AKT2,0.274600,0.022383,endo,AKT2,1.119152
BAD,-0.032200,0.434024,endo,BAD,21.701189
BECN1,-0.023561,0.900859,endo,BECN1,45.042969
PIK3C3,0.125200,0.012287,endo,PIK3C3,0.614332
AKT2,0.181700,0.829271,brca,AKT2,41.463536
BAD,1.169350,0.041521,brca,BAD,2.076072


In [88]:
plotCircleHeatMap(selected,"P_Value","Difference_In_Median","comparison","cancer_type")

/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Loading BokehJS ...

In [93]:
plotCircleHeatMap(selected,"P_Value","P_Value","comparison","cancer_type")

/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Loading BokehJS ...

In [67]:
import cptac.utils as u

/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [78]:
interacting = u.get_interacting_proteins("PIK3CA")
len(interacting)

36

In [73]:
df.head()

,Difference_In_Median,P_Value,cancer_type,comparison,size
A2ML1,NaN,NaN,colon,A2ML1,NaN
AADAT,NaN,NaN,colon,AADAT,NaN
AAGAB,-0.1390,0.039189,colon,AAGAB,1.959434
AASDHPPT,0.0628,0.973515,colon,AASDHPPT,48.675735
AATF,-0.0250,0.568821,colon,AATF,28.441045


In [76]:
inter = df[df.index.isin(interacting)]

In [90]:
plotCircleHeatMap(inter,"P_Value","Difference_In_Media","comparison","cancer_type")

/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Loading BokehJS ...

In [94]:
plotCircleHeatMap(inter,"P_Value","P_Value","comparison","cancer_type")

/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Loading BokehJS ...

In [97]:
plotCircleHeatMap(selected,circle_var="Difference_In_Median", color_var="P_Value", x_axis="cancer_type",y_axis="comparison")

/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Loading BokehJS ...